# INT8 acceleration of semantic segmentation with NNCF/POT for OpenVINO

This notebook runs through the process of enabling [NNCF](https://github.com/openvinotoolkit/nncf) in a simple semantic segmentation pipeline for training UNet on Camvid, and comparing results against the Post-Training Optimization Tool ([POT](https://docs.openvinotoolkit.org/latest/pot_README.html))'s quantization.

> NOTE: _For this notebook to function, please make sure that your Python environment has `openvino`, `openvino-dev` and `nncf[torch]` packages installed.

In [ ]:
import sys
python = sys.executable
%ls

Get the dataset:

In [ ]:
!git clone https://github.com/alexgkendall/SegNet-Tutorial data

### Obtaining the uncompressed (FP32) performance and accuracy baselines
Download the FP32 pre-trained weights checkpoint for UNet on CamVid:

In [ ]:
!curl -o unet_camvid.pth https://storage.openvinotoolkit.org/repositories/nncf/openvino_notebook_ckpts/303_unet_camvid.pth

Measure the baseline FP32 accuracy in PyTorch and produce an ONNX for future OpenVINO ingestion:

In [ ]:
!$python main.py -m test export --resume unet_camvid.pth --data data/CamVid -b 1 --to-onnx unet_camvid.onnx

Evaluate the FP32 model on OpenVINO (accuracy and performance); first, convert the ONNX file to the intermediate representation (IR) using the [Model Optimizer](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)

In [ ]:
!mo --input_model unet_camvid.onnx --mean_values [99.603,103.329,105.6567] --scale_values [75.643,77.821,76.746] --reverse_input_channels

Measure the accuracy (mIoU metric) on the target dataset using the Accuracy Checker tool (part of the `openvino-dev` package with the prepared .yml specification of the dataset.

In [ ]:
!accuracy_check -c unet_camvid.yml -m unet_camvid.xml

For measuring performance, we use the [Benchmark Tool](https://docs.openvinotoolkit.org/latest/openvino_inference_engine_tools_benchmark_tool_README.html) - OpenVINO's inference performance measurement tool.

In [ ]:
!benchmark_app -m unet_camvid.xml

### POT approach
Use POT to obtain an INT8 model. Note that this path requires your dataset to be supported by the Accuracy Checker tool (part of the `openvino-dev` Python package), which is not true for all datasets.

In [ ]:
!pot -q accuracy_aware --max-drop 0.01 -m unet_camvid.xml -w unet_camvid.bin --ac-config unet_camvid.yml --output-dir pot_int8 --name unet_camvid_pot_aa_int8 -e -d

Measure performance for the INT8-POT model (the accuracy results are already visible in the POT output above) 

In [ ]:
!benchmark_app -m pot_int8/optimized/unet_camvid_pot_aa_int8.xml

### NNCF approach

Make sure that NNCF is installed in your Python environment - either from the source repository or the PyPI using `pip install nncf` - then integrate NNCF into the training pipeline. The line below apples the patch to allow for producing NNCF-compressed INT8 models. 5 lines of code (excluding import statements) and a simple .json config is enough for this integration.

In [ ]:
!patch -p1 < nncf.patch
!cat nncf.patch

Perform compression-aware fine-tuning using NNCF, starting from the regular PyTorch checkpoint (unet_camvid.pth), for 10 epochs, picking the best result and exporting it into an INT8 ONNX file (unet_camvid_int8.onnx). The training takes about 10 minutes on a single NVIDIA RTX 2080 Ti GPU. Note that there is no requirement for the support of the dataset by a third-party tool in order to obtain the results since the dataset is already supported by the source training pipeline and NNCF integrates into the pipeline seamlessly.

In [ ]:
!$python main.py -m train test export --resume unet_camvid.pth --data data/CamVid -b 3 --epochs 10 --to-onnx unet_camvid_int8.onnx --nncf_config unet_camvid_int8.json

Convert the NNCF-INT8 ONNX file into the NNCF-INT8 IR for OpenVINO ingestion

In [ ]:
!mo --input_model unet_camvid_int8.onnx --mean_values [99.603,103.329,105.6567] --scale_values [75.643,77.821,76.746] --reverse_input_channels

Evaluate the NNCF-INT8 model in OpenVINO, accuracy and performance-wise:

In [ ]:
!accuracy_check -c unet_camvid.yml -m unet_camvid_int8.xml

In [ ]:
!benchmark_app -m unet_camvid_int8.xml